In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import * 

# Without Bias

In [2]:
X_train = [[-1,-1] for x in range(10)]+[[1,1] for x in range(10)]+[[-1,1] for x in range(10)]+[[1,-1] for x in range(10)]
X_test =  [[-1,-1] for x in range(10)]+[[1,1] for x in range(10)]+[[-1,1] for x in range(10)]+[[1,-1] for x in range(10)]
y_train = [0 for x in range(20)]+[1 for x in range(20)]
y_test =  [0 for x in range(20)]+[1 for x in range(20)]

In [3]:
with open('results/version7/train_xor.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version7/test_xor.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)

### Without Bias

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput, epoch_errosPhaseEncoding  = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=40, 
                                                                                                 trainingBias=False,
                                                                                                 error_by_epoch = True,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

In [ ]:
with open('results/version6/error_by_epoch_unbiased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosPhaseEncoding': epoch_errosPhaseEncoding, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [ ]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS = experiment_TRAIN(X_train, 
                                                                                                y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=20, 
                                                                                                 trainingBias=False,
                                                                                                 error_by_epoch = False,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

In [ ]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS]

In [ ]:
with open('results/version7/weights_unbiased_v7.json', 'w+') as fp:
    json.dump(weights, fp)

#### Test

In [ ]:
with open('results/version7/weights_unbiased_v7.json') as json_file:
    weights = json.load(json_file)

In [ ]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              repeat=10,
                              bias=False, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

In [ ]:
with open('results/version7/experiments_unbiased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)

## BIAS

In [ ]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput, epoch_errosPhaseEncoding  = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=40, 
                                                                                                 trainingBias=True,
                                                                                                 error_by_epoch = True,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})
                                    

In [ ]:
with open('results/version6/error_by_epoch_biased.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosPhaseEncoding': epoch_errosPhaseEncoding, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [3]:
weightsEncodingWeight, weightsEncodingInput, weightsPhaseEncoding, weightsHSGS = experiment_TRAIN(X_train, 
                                                                                                 y_train,
                                                                                                 lrParameter=0.2, 
                                                                                                 n_epochs=20, 
                                                                                                 trainingBias=True,
                                                                                                 error_by_epoch = False,
                                                                                                 trainingApproaches={'neuronio-classico', 
                                                                                                                     'neuronio-classico-bin', 
                                                                                                                     'hsgs',
                                                                                                                     'encoding-weight', 
                                                                                                                     'encoding-input',
                                                                                                                     'phase-encoding'})

[[1.5707963267948968], [-1.5707963267948963, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[1.4036006757023898], [-1.28142632868209, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.8953035914174722], [-0.6928257246252647, 1.5707963267948963]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.2834984112099861], [-0.20216123709859823, 1.5707963267948968]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.18750386597690621], [0.1330732419339051, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.4782059148013804], [0.34646770831974216, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.6550052596847331], [0.48520607904032936, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[0.7766448419437949], [0.5869366764676983, 7.853981633974483]]
[[1.5707963267948968], [1.5707

In [4]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsPhaseEncoding, 
          weightsHSGS]

In [5]:
with open('results/version7/weights_biased_v7.json', 'w+') as fp:
    json.dump(weights, fp)

In [6]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              repeat=10,
                              bias=True, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'phase-encoding','encoding-input'})

[[2.286156793444583], [7.853981633974483, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.29629923034736], [4.71238898038469, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.286156793444583], [7.853981633974483, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.29629923034736], [4.71238898038469, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.286156793444583], [7.853981633974483, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.29629923034736], [4.71238898038469, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.286156793444583], [7.853981633974483, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]
[[2.29629923034736], [4.71238898038469, 7.853981633974483]]
[[1.5707963267948968], [1.5707963267948963, 7.853981633974483]]


In [7]:
with open('results/version7/experiments_biased.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)